# Add Features from SDE into Feature Service

The script utilizes both Esri's Destkop Python library 'arcpy' & the web Python API.  Connect to ArcGIS Online or Enterprise
parse through various datapoints and add them into a service.

In [1]:
# Import modules
import json
import arcpy
import getpass
import time
from datetime import datetime
from dateutil import tz
from arcpy import env
from arcgis.gis import *

## Connect to ArcGIS Online - Python API
Estbalish connection, search for content by name to return the item's ID.  Once found, connect to relevant index dataset, and truncate the service in preparation for data load.

In [2]:
# Utilize getpass for hiding password for demonstration purposes
password = getpass.getpass()

········


In [3]:
# Establish connection to a WebGIS organization
# ArcGIS Online
gis = GIS('http://www.arcgis.com', 'abrown_citygov', password)

# For ArcGIS Enterprise, connect as "https:<server DNS>/<web adaptor for portal>"
# Example: GIS('http://neenterprise.esri.com/portal', 'username', password)  

In [4]:
# Search by title
search = gis.content.search(query="title: PhillyCrimeExample", item_type="Feature Service")
search

[<Item title:"PhillyCrimeExample" type:Feature Layer Collection owner:abrown_citygov>]

In [5]:
# From search list, return the index for service one is interseted in connecting to
item = search[0]
itemID = item.id
itemID

'1d1ee09e66084db0a189ee0e86ba9689'

In [6]:
# Establish connection to Feature Layer item
feature_layer_item = gis.content.get(itemID)
flayers = feature_layer_item.layers
flayer = flayers[0]
print(flayer)
print(type(flayer))

<FeatureLayer url:"https://services6.arcgis.com/0p6i4J6xhQas4Unf/arcgis/rest/services/PhillyCrimeExample/FeatureServer/0">
<class 'arcgis.features.layer.FeatureLayer'>


In [8]:
# Truncate the dataset, cannot be utlized on layers with synch
flayer.manager.truncate()

# Alternative Option, utilize delete_features method in class FeatureLayer
# flayer.delete_features(where="1=1")

# Return total number of records after insert
total_result = flayer.query(where='1=1', out_fields='*')
print(len(total_result))

0


## Connect to Enterprise Geodatabase

Establish connection to enterprise geodatabase.  List feature classes, locate dataset of interest. Manipulate the field list to add formalized shape and object id columns.  Create a search cursor on the dataset and create emtpy dictionaries & lists for data preparation.

In [9]:
# Establish arcpy desktop workspace
wspace = r'C:\Users\alex8694\AppData\Roaming\ESRI\Desktop10.6\ArcCatalog\Enterprise_SDE.sde'
env.workspace = wspace

In [10]:
# List Feature classes within geodatabase
fclist = arcpy.ListFeatureClasses()

In [11]:
# Find correct dataset
for item in fclist:
    if item == 'Enterprise.SDE.PhillyCrimeSubset':
        data = item 

In [12]:
# List fields to create a field key
fieldlist = arcpy.ListFields(data)
fnames = []
for field in fieldlist:
    fnames.append(field.name)
    
# Remove specific fields for shape and objectid and add arcgis acceptable field values.
fnames.remove('Shape')
fnames.remove('OBJECTID_1')
fnames.append('SHAPE@')
fnames.insert(0,"OID@")
fnames

['OID@',
 'point_y',
 'objectid',
 'dc_dist',
 'psa',
 'dispatch_d',
 'dispatch_1',
 'hour_',
 'dc_key',
 'location_b',
 'ucr_genera',
 'text_gener',
 'point_x',
 'dispatch_t',
 'SHAPE@']

In [13]:
# Establish search cursor to read input data
cursor = arcpy.da.SearchCursor(data, fnames)

# Create placeholder diciontary and lists for data manipulation
attribute_dict = {}
geom_dict = {}
final_features = []

## Prepare data for insert into Feature Service

Manipulate the data coming from arcpy insert cursor, be aware of your time zone properties, add to a service.

In [15]:
# Auto Detect Time Zones for time conversion
# Utilizing data from local time, be careful with your datetimes!
from_zone = tz.tzlocal()

# From search cursor, construct dictionaries for attributes and geometry
for row in cursor:
    attribute_dict[cursor.fields[0]] = row[0]
    attribute_dict[cursor.fields[1]] = row[1]
    attribute_dict[cursor.fields[2]] = row[2]
    attribute_dict[cursor.fields[3]] = row[3]
    attribute_dict[cursor.fields[4]] = row[4]
    
    # Date & Time are not calculated properly.  Date is in datetime, but time is a string.
    # Reconstruct full string with proper date times
    
    # Thanks to audience member at GeoDev who made me realize not to code late at night!
    # dt_only = row[5].strftime("%Y-%m-%d %H:%M:%S")[10:]
    dt_only = row[5].strftime("%Y-%m-%d")
    dt_string = dt_only + ' ' + row[13]
    datetime_object = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")

    # Tell the datetime object that it's in EST time zone since datetime objects are 'naive' by default
    eastern = datetime_object.replace(tzinfo=from_zone)
    
    # Convert datetime object into unix timestamp, set as field equivalent
    attribute_dict[cursor.fields[5]] = time.mktime(eastern.timetuple()) * 1e3 + eastern.microsecond / 1e3  
    
    # Continue adding data to proper field mappings
    attribute_dict[cursor.fields[6]] = row[6]
    attribute_dict[cursor.fields[7]] = row[7]
    attribute_dict[cursor.fields[8]] = row[8]
    attribute_dict[cursor.fields[9]] = row[9]
    attribute_dict[cursor.fields[10]] = row[10]
    attribute_dict[cursor.fields[11]] = row[11]
    attribute_dict[cursor.fields[12]] = row[12]
    attribute_dict[cursor.fields[13]] = row[13]
    
    # Write geometry to separate dictionary
    for pnt in row[14]:
        geom_dict['x'] = pnt.X
        geom_dict['y'] = pnt.Y
        
    # Convert dictionaries into a JSON String 
    att = json.dumps(attribute_dict)
    geo = json.dumps(geom_dict)
    
    # Manipulate the string to reconstruct proper JSON
    att.lstrip('{') 
    att = '{"attributes":' + att 
    geo.lstrip('{')
    geo = ',"geometry": ' + geo + '}'
     
    # Add the two json strings together, convert back to diciontary and load into list to prepare for data insert into 
    # service.
    concat = att + geo
    data_dict = json.loads(concat)
    final_features.append(data_dict)
    print(final_features)
    
    # Add data to Feature Service
    flayer.edit_features(adds=final_features, updates=None, deletes=None, gdb_version=None, use_global_ids=False, rollback_on_failure=True)
    
    # Clear the final list
    final_features = []
    
# Delete in memory cursor on database table
del cursor

[{'attributes': {'OID@': 2033, 'point_y': 40.06261422, 'objectid': 8007, 'dc_dist': '07', 'psa': '1', 'dispatch_d': 1428647040000.0, 'dispatch_1': '2015-04-10', 'hour_': 2, 'dc_key': '201507009700', 'location_b': '8400 BLOCK E ROOSEVELT BLVD', 'ucr_genera': '500', 'text_gener': 'Burglary Residential', 'point_x': -75.04307602, 'dispatch_t': '02:24:00'}, 'geometry': {'x': -75.04307600994986, 'y': 40.06261422027012}}]
[{'attributes': {'OID@': 4518, 'point_y': 40.06313326, 'objectid': 23185, 'dc_dist': '08', 'psa': '1', 'dispatch_d': 1442591400000.0, 'dispatch_1': '2015-09-18', 'hour_': 11, 'dc_key': '201508035817', 'location_b': '8300 BLOCK E ROOSEVELT BLVD', 'ucr_genera': '1100', 'text_gener': 'Fraud', 'point_x': -75.04218487, 'dispatch_t': '11:50:00'}, 'geometry': {'x': -75.04218486993574, 'y': 40.06313326039168}}]
[{'attributes': {'OID@': 4713, 'point_y': 40.06398515, 'objectid': 17890, 'dc_dist': '08', 'psa': '1', 'dispatch_d': 1440009480000.0, 'dispatch_1': '2015-08-19', 'hour_': 14,

[{'attributes': {'OID@': 33386, 'point_y': 40.06131457, 'objectid': 227991, 'dc_dist': '08', 'psa': 'L', 'dispatch_d': 1146880440000.0, 'dispatch_1': '2006-05-05', 'hour_': 21, 'dc_key': '200608019563', 'location_b': '8300 BLOCK ROOSEVELT BLVD', 'ucr_genera': '800', 'text_gener': 'Other Assaults', 'point_x': -75.0429753, 'dispatch_t': '21:54:00'}, 'geometry': {'x': -75.04297530026986, 'y': 40.06131457041113}}]
[{'attributes': {'OID@': 34583, 'point_y': 40.06131457, 'objectid': 218663, 'dc_dist': '08', 'psa': 'L', 'dispatch_d': 1158507480000.0, 'dispatch_1': '2006-09-17', 'hour_': 11, 'dc_key': '200608041771', 'location_b': '8300 BLOCK ROOSEVELT BLVD', 'ucr_genera': '800', 'text_gener': 'Other Assaults', 'point_x': -75.0429753, 'dispatch_t': '11:38:00'}, 'geometry': {'x': -75.04297530026986, 'y': 40.06131457041113}}]
[{'attributes': {'OID@': 35009, 'point_y': 40.06131457, 'objectid': 219619, 'dc_dist': '08', 'psa': 'L', 'dispatch_d': 1138906080000.0, 'dispatch_1': '2006-02-02', 'hour_':

[{'attributes': {'OID@': 67962, 'point_y': 40.06100237, 'objectid': 424688, 'dc_dist': '07', 'psa': 'C', 'dispatch_d': 1190619960000.0, 'dispatch_1': '2007-09-24', 'hour_': 3, 'dc_key': '200707032493', 'location_b': '8300 BLOCK E ROOSEVELT BLVD', 'ucr_genera': '600', 'text_gener': 'Thefts', 'point_x': -75.04379661, 'dispatch_t': '03:46:00'}, 'geometry': {'x': -75.04379660972575, 'y': 40.061002359970985}}]
[{'attributes': {'OID@': 70271, 'point_y': 40.06131457, 'objectid': 485063, 'dc_dist': '08', 'psa': 'L', 'dispatch_d': 1175154720000.0, 'dispatch_1': '2007-03-29', 'hour_': 3, 'dc_key': '200708013202', 'location_b': '8300 BLOCK ROOSEVELT BLVD', 'ucr_genera': '2100', 'text_gener': 'DRIVING UNDER THE INFLUENCE', 'point_x': -75.0429753, 'dispatch_t': '03:52:00'}, 'geometry': {'x': -75.04297530026986, 'y': 40.06131457041113}}]
[{'attributes': {'OID@': 71319, 'point_y': 40.06313326, 'objectid': 455791, 'dc_dist': '08', 'psa': 'L', 'dispatch_d': 1188391860000.0, 'dispatch_1': '2007-08-29', 

In [16]:
# Return total number of records after insert
final_result = flayer.query(where='1=1', out_fields='*')
print(len(final_result))

55
